In [1]:
import re
import string
import numpy
import torch.nn as nn
import torch

In [2]:
text = '''a man and a woman received mostly positive reviews gender identity refers to an individual personal sense of identity as man or               woman man and woman arrested after male attacked a man and a woman both aged in their thirty in nature man and woman i examine the               representation of men and women by focussing on the grammatical behaviour of the noun lemmas man and woman examined the frequency with               the words man men and woman women bachelor and spinster extend to basic terms for male and female human beings she looked at man woman               and boy girl and found that negative words are used more frequently with woman girl than with man boy the four directions are the               directions north east south and west appears to rotate around an axis passing through the north and south poles of the earth in these               locations whether the sun is moving from east to west through north or south by watching its movements the sun rises either north or               south of true east and sets north or south of true west for all locations the sun is seen to rise north of east and set north of west               analogue watch can be used to locate north and south this axis intersects the sphere at the north and south poles which appear to the               observer to lie directly above due north and south respectively on the horizon morning precedes afternoon evening and night in the               sequence of a day also true for evening and night night or nighttime is the period of darkness from sunset to sunrise unlike good               morning good afternoon and good evening good night is not used as a greeting what are the time ranges for morning afternoon evening               night and do they change with location or the seasons empire was ruled by an emperor king an emperor was also king grand prince was               reserved for an emperor or king grand prince in some languages is title before emperor and king start with the king or queen despite               they were an emperor or empress a queen regnant is a female monarch equivalent in rank to a king at the top of the hierarchy of royal               titles is the king and queen the relatives of the king and queen known as prince and princess only last royal titles work what the king               and queen actually do prince became a king a prince the male child of the king limited palette consisting of red yellow black and white               a mixture of yellow and black will appear as a variety of green basic colors include black white red green blue and yellow set of twelve               lack gray white pink red orange yellow green blue nonspectral colors are the colors gray black and white the antonym of white is black               black and white also appealed to architects tied with black and white black and white often represent the contrast between light and               darkness day and night male and female calendar are generally recognized spring summer autumn or fall and winter four seasons winter               spring summer autumn fall in the middle of summer and winter number of seasons between summer and winter can number from one to three               universally translated as spring summer autumn and winter but actually begin much earlier autumn is the season between summer and winter               lasting from march to june or from september to december summer is the season between spring and autumn from june to september or from               december to march winter spring summer and autumn are the seasons of the year december january and february are the winter months march               april and may are the spring months june july and august are the summer months day to learn about days weeks months and years twelve months               ake year seven days make week days fit into a week into a month and into a year each new day starts at midnight days in some months to make               them all add up to one year'''

SkipGram

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Создаем веса для входного и выходного слоя
        self.in_embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.out_embedding = nn.Embedding(self.vocab_size, self.embedding_dim)

        # Инициализируем веса
        self.in_embedding.weight.data.uniform_(-1, 1)
        self.out_embedding.weight.data.uniform_(-1, 1)

    def forward(self, input_word, output_word):
        # Получаем эмбеддинги для входного и выходного слов
        input_emb = self.in_embedding(input_word)
        output_emb = self.out_embedding(output_word)

        # Умножаем эмбеддинги и берем сумму по последней оси
        dot_product = torch.sum(input_emb * output_emb, dim=-1)

        # Применяем логистическую функцию
        score = nn.functional.logsigmoid(dot_product)

        return score


    def get_word_embedding(self, word, word_to_idx):
        word = Variable(torch.LongTensor([word_to_idx[word]]))
        return self.out_embedding(word).view(1, -1)

def process_text(text: str):

    tokens = [word for word in text.split(' ') if word != '']

    # Создаем словарь и преобразуем токены в индексы
    vocab = set(tokens)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for idx, word in enumerate(vocab)}
    
    data = [(torch.tensor(word_to_idx[tokens[i]]), torch.tensor(word_to_idx[tokens[j]]))
            for i in range(len(tokens)) for j in range(max(0, i-2), min(len(tokens), i+3)) if i != j]

    return data, word_to_idx, idx_to_word


def train(data: str):
    num_epochs = 13

    data, word_to_idx, idx_to_word = process_text(data)

    # Определяем параметры модели
    vocab_size = len(word_to_idx)
    embedding_dim = 100
    # Создаем экземпляр модели
    model = SkipGram(vocab_size, embedding_dim)
    # Определяем функцию потерь и оптимизатор
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.025)

    for epoch in range(num_epochs):
        total_loss = 0

        # Обходим датасет
        for center_word, context_word in data:
            # Обнуляем градиенты
            optimizer.zero_grad()

            # Получаем предсказание и вычисляем функцию потерь
            score = model(center_word, context_word)
            loss = criterion(score, torch.ones_like(score))

            # Вычисляем градиенты и обновляем веса
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Выводим функцию потерь после каждой эпохи
        print("Epoch {}, Loss: {:.4f}".format(epoch+1, total_loss/len(data)))
    return {
        word: model.get_word_embedding(word, word_to_idx).detach().numpy() for word in word_to_idx.keys()
    }


In [4]:
res = train(text)

Epoch 1, Loss: 1.6972
Epoch 2, Loss: 1.2065
Epoch 3, Loss: 0.9881
Epoch 4, Loss: 0.8747
Epoch 5, Loss: 0.8146
Epoch 6, Loss: 0.7822
Epoch 7, Loss: 0.7634
Epoch 8, Loss: 0.7513
Epoch 9, Loss: 0.7428
Epoch 10, Loss: 0.7365
Epoch 11, Loss: 0.7317
Epoch 12, Loss: 0.7278
Epoch 13, Loss: 0.7246


In [5]:
text = '''a man and a woman received mostly positive reviews gender identity refers to an individual personal sense of identity as man or               woman man and woman arrested after male attacked a man and a woman both aged in their thirty in nature man and woman i examine the               representation of men and women by focussing on the grammatical behaviour of the noun lemmas man and woman examined the frequency with               the words man men and woman women bachelor and spinster extend to basic terms for male and female human beings she looked at man woman               and boy girl and found that negative words are used more frequently with woman girl than with man boy the four directions are the               directions north east south and west appears to rotate around an axis passing through the north and south poles of the earth in these               locations whether the sun is moving from east to west through north or south by watching its movements the sun rises either north or               south of true east and sets north or south of true west for all locations the sun is seen to rise north of east and set north of west               analogue watch can be used to locate north and south this axis intersects the sphere at the north and south poles which appear to the               observer to lie directly above due north and south respectively on the horizon morning precedes afternoon evening and night in the               sequence of a day also true for evening and night night or nighttime is the period of darkness from sunset to sunrise unlike good               morning good afternoon and good evening good night is not used as a greeting what are the time ranges for morning afternoon evening               night and do they change with location or the seasons empire was ruled by an emperor king an emperor was also king grand prince was               reserved for an emperor or king grand prince in some languages is title before emperor and king start with the king or queen despite               they were an emperor or empress a queen regnant is a female monarch equivalent in rank to a king at the top of the hierarchy of royal               titles is the king and queen the relatives of the king and queen known as prince and princess only last royal titles work what the king               and queen actually do prince became a king a prince the male child of the king limited palette consisting of red yellow black and white               a mixture of yellow and black will appear as a variety of green basic colors include black white red green blue and yellow set of twelve               lack gray white pink red orange yellow green blue nonspectral colors are the colors gray black and white the antonym of white is black               black and white also appealed to architects tied with black and white black and white often represent the contrast between light and               darkness day and night male and female calendar are generally recognized spring summer autumn or fall and winter four seasons winter               spring summer autumn fall in the middle of summer and winter number of seasons between summer and winter can number from one to three               universally translated as spring summer autumn and winter but actually begin much earlier autumn is the season between summer and winter               lasting from march to june or from september to december summer is the season between spring and autumn from june to september or from               december to march winter spring summer and autumn are the seasons of the year december january and february are the winter months march               april and may are the spring months june july and august are the summer months day to learn about days weeks months and years twelve months               ake year seven days make week days fit into a week into a month and into a year each new day starts at midnight days in some months to make               them all add up to one year'''

CBOW

In [6]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(CBOW, self).__init__()
        
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_size)
        self.activation_function1 = nn.ReLU()

        self.linear2 = nn.Linear(hidden_size, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out
    
    def get_word_embedding(self, word, word_to_idx):
        word = Variable(torch.LongTensor([word_to_idx[word]]))
        return self.embeddings(word).view(1, -1)

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def create_cbow_dataset(text):
    data = []
    for i in range(4, len(text) - 4):
        context = [text[i - 4], text[i - 3], text[i - 2], text[i - 1],
                   text[i + 1], text[i + 2], text[i + 3], text[i + 4]]
        target = text[i]
        data.append((context, target))
    return data

def train(data: str = ''):

    embd_size = 150

    learning_rate = 0.01
    n_epoch = 25

    text = [word for word in data.split(' ') if word != '']

    vocab = set(text)
    vocab_size = len(vocab)

    word_to_idx = {w: i for i, w in enumerate(vocab)}
    idx_to_word = {i: w for i, w in enumerate(vocab)}

    cbow_train = create_cbow_dataset(text)

    hidden_size = 256

    loss_fn = nn.NLLLoss()

    model = CBOW(vocab_size, embd_size, hidden_size)

    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(n_epoch):
        total_loss = 0
        for context, target in cbow_train:
            ctx_idxs = [word_to_idx[w] for w in context]
            ctx_var = Variable(torch.LongTensor(ctx_idxs))

            model.zero_grad()
            log_probs = model(ctx_var)

            loss = loss_fn(log_probs, Variable(
                torch.LongTensor([word_to_idx[target]])))

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Выводим функцию потерь после каждой эпохи
        print("Epoch {}, Loss: {:.4f}".format(epoch+1, total_loss/len(data)))

    context = ['autumn', 'spring']
    context_vector = make_context_vector(context, word_to_idx)
    a = model(context_vector)
    print(f'Context: {context}\n')
    print(f'Prediction: {idx_to_word[torch.argmax(a[0]).item()]}')

    return {
        word: model.get_word_embedding(word, word_to_idx).detach().numpy() for word in word_to_idx.keys()
    }


In [7]:
words = train(text)

Epoch 1, Loss: 0.8539
Epoch 2, Loss: 0.5885
Epoch 3, Loss: 0.3886
Epoch 4, Loss: 0.2157
Epoch 5, Loss: 0.1029
Epoch 6, Loss: 0.0694
Epoch 7, Loss: 0.0454
Epoch 8, Loss: 0.0180
Epoch 9, Loss: 0.0093
Epoch 10, Loss: 0.0049
Epoch 11, Loss: 0.0036
Epoch 12, Loss: 0.0029
Epoch 13, Loss: 0.0025
Epoch 14, Loss: 0.0023
Epoch 15, Loss: 0.0021
Epoch 16, Loss: 0.0020
Epoch 17, Loss: 0.0018
Epoch 18, Loss: 0.0017
Epoch 19, Loss: 0.0017
Epoch 20, Loss: 0.0016
Epoch 21, Loss: 0.0015
Epoch 22, Loss: 0.0015
Epoch 23, Loss: 0.0014
Epoch 24, Loss: 0.0014
Epoch 25, Loss: 0.0013
Context: ['autumn', 'spring']

Prediction: fall
